In [ ]:
from hyperpyyaml import load_hyperpyyaml
import os
from cosyvoice.cli.frontend import CosyVoiceFrontEnd
from cosyvoice.cli.model import CosyVoiceModel, CosyVoice2Model, CosyVoice3Model

hyper_yaml_path = '/home/ecs-user/code/happen/A100-04/happen/AUDIO/HydraVox/pretrained_models/Fun-CosyVoice3-0.5B/cosyvoice3_mtp.yaml'
model_dir = "/home/ecs-user/code/happen/A100-04/happen/AUDIO/HydraVox/pretrained_models/Fun-CosyVoice3-0.5B"

In [ ]:
with open(hyper_yaml_path, 'r') as f:
    configs = load_hyperpyyaml(f, overrides={'qwen_pretrain_path': os.path.join(model_dir, 'CosyVoice-BlankEN')})

In [ ]:
sample_rate = configs['sample_rate']

In [ ]:
frontend = CosyVoiceFrontEnd(configs['get_tokenizer'],
                                    configs['feat_extractor'],
                                    '{}/campplus.onnx'.format(model_dir),
                                    '{}/speech_tokenizer_v3.onnx'.format(model_dir),
                                    '{}/spk2info.pt'.format(model_dir),
                                    configs['allowed_special'])

In [ ]:
model = CosyVoice3Model(configs['llm'], configs['flow'], configs['hift'], True)

In [ ]:
model.load('{}/llm_multihead_10k.pt'.format(model_dir),
                '{}/flow.pt'.format(model_dir),
                '{}/hift.pt'.format(model_dir))

In [ ]:
prompt_text = frontend.text_normalize("同学们，今天的课堂从一个简单的问题开始", split=False, text_frontend=True)
model_input = frontend.frontend_zero_shot("A的平方加上B的平方,再开方, 余是多少? 你能算一算吗？", prompt_text, "/home/ecs-user/code/happen/A100-04/happen/AUDIO/HydraVox/outputs/emotion_batch/hanxue_prompt.wav", sample_rate, '')
output= model.tts(**model_input, stream=False, speed=1.0)

In [ ]:
from IPython.display import Audio
Audios = []
for item in output:
    Audios.append(item['tts_speech'])

In [ ]:
Audios

In [ ]:
import numpy as np
Audio(Audios[0].cpu().numpy(), rate=sample_rate)